In [1]:
# -*- coding: utf-8 -*-
"""
Created on June 7, 2024
Last modified on June 7, 2024
@Author: Guan-Fu Liu

To generate the yield file used in ChemEvoIMF
"""
import numpy as np
import pandas as pd
import os
import sys
import re
import h5py
sys.path.insert(0, '../')
import utils

# Input file format
## Yields file
It should be named by `Z=X.XXXX-Original.csv`.
The yields are in the unit of solar mass.
## Mass lifetime file
It should be named by `Z=X.XXXX-MassLifetime.csv`.
The first column is the initial mass in solar mass and the second column is the lifetime in yr.
Mass lifetime file is not a must, since there is a built-in mass lifetime relation.
However, if you mass lifetime files, they should correspond to the yields files.

# Output HDF5 file
There are two kinds of HDF5 files: `yields1.h5` and `yields2.h5`.
They are equivalent, but the format is different.

In [2]:
file_list = os.listdir()
file_list = [f for f in file_list if f.endswith('.csv')]
Z_list = [file.split("-")[0] for file in file_list]
Z_list = list(set(Z_list))

In [3]:
with h5py.File("yields1.h5", "w") as f:
    for Z in Z_list:
        f.create_group(Z)
        df1 = pd.read_csv(f"{Z}-Original.csv")
          # hdf5 does not support string with object type
        # "|S" will find the maximum length of the string in the selected column
        df1['M'] = df1['M'].astype('|S')
        data = df1.to_records(index=False)
        dtype = df1.to_records(index=False).dtype
        f[Z].attrs['Z'] = "%s" % Z[2:]
        f[Z].attrs['MassUnit'] = 'Msun'
        f[Z].create_dataset('Original', data=data, dtype=dtype)
        df1 = pd.read_csv(f"{Z}-Original.csv", index_col=0)
        columns = ["%0.6e" % a for a in np.logspace(np.log10(0.08), np.log10(150), 300)]
        df2 = pd.DataFrame(np.zeros((len(df1.index), len(columns))), index=df1.index, columns=columns)
        for i, row in df1.iterrows():
            x1 = row.index.to_numpy().astype(float)  # x1 is the initial mass from original yields
            y1 = row.to_numpy().astype(float)  # y1 is the remnant mass from original yields
            # extrapolate and interpolate the yields
            df2.loc[row.name, :] = [utils.extra_interpolate_yields(x1, y1, row.name, float(col)) for col in columns]
        df2.reset_index(drop=True, inplace=True)
        df2.to_csv(f"{Z}-Interpolated.csv", index=False)
        data = df2.to_records(index=False)
        dtype = df2.to_records(index=False).dtype
        f[Z].create_dataset('Interpolated', data=data, dtype=dtype)
        f[Z].create_dataset('MassLifetime', data=np.loadtxt(f"{Z}-MassLifetime.csv", delimiter=","))

In [5]:
with h5py.File("yields2.h5", 'a') as f:
    for Z in Z_list:
        f[Z.replace("=", "_").replace(".", "_")].create_dataset('MassLifetime', 
                                                                data=np.loadtxt(f"{Z}-MassLifetime.csv", delimiter=","))
        f[Z.replace("=", "_").replace(".", "_")].attrs['Z'] = "%s" % Z[2:]
        f[Z.replace("=", "_").replace(".", "_")].attrs['MassUnit'] = 'Msun'
for Z in Z_list:
    df1 = pd.read_csv(f"{Z}-Original.csv", index_col=0)
    df1.to_hdf('yields2.h5', key='/%s/Original' % Z.replace("=", "_").replace(".", "_"), mode='a')
    df2 = pd.read_csv(f"{Z}-Interpolated.csv", index_col=0)
    df2.to_hdf('yields2.h5', key='/%s/Interpolated' % Z.replace("=", "_").replace(".", "_"), mode='a')
